In [45]:
%load_ext lab_black
import argparse
import contextlib
import datetime
import io
import logging
import multiprocessing
import os
import random
import sys
from itertools import chain, combinations
from timeit import default_timer as timer

import altair as alt
import altair_viewer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import peewee
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from json_tricks import dumps, loads
from playhouse.shortcuts import model_to_dict
from scipy.stats import randint, uniform
from sklearn.datasets import load_iris
from tabulate import tabulate
from IPython.core.display import display, HTML

from active_learning.cluster_strategies import (
    DummyClusterStrategy,
    MostUncertainClusterStrategy,
    RandomClusterStrategy,
    RoundRobinClusterStrategy,
)
from active_learning.dataStorage import DataStorage
from active_learning.experiment_setup_lib import (
    ExperimentResult,
    classification_report_and_confusion_matrix,
    get_db,
    get_single_al_run_stats_row,
    get_single_al_run_stats_table_header,
)
from active_learning.sampling_strategies import (
    BoundaryPairSampler,
    CommitteeSampler,
    RandomSampler,
    UncertaintySampler,
)

alt.renderers.enable("altair_viewer")
#  alt.renderers.enable('vegascope')

config = {
    "datasets_path": "../datasets",
    "db": "tunnel",
    "param_list_id": "best_global_score",
}

db = get_db(db_name_or_type=config["db"])

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [46]:
# select count(*), dataset_name from experimentresult group by dataset_name;
results = ExperimentResult.select(
    ExperimentResult.dataset_name,
    peewee.fn.COUNT(ExperimentResult.id_field).alias("dataset_name_count"),
).group_by(ExperimentResult.dataset_name)

for result in results:
    print("{:>4,d} {}".format(result.dataset_name_count, result.dataset_name))

184,746 dwtc
4,942 hiva
4,956 ibn_sina
4,933 orange
4,901 sylva
4,871 zebra


In [47]:
#  SELECT param_list_id, avg(fit_score), stddev(fit_score), avg(global_score), stddev(global_score), avg(start_set_size) as sss, count(*) FROM experimentresult WHERE start_set_size = 1 GROUP BY param_list_id ORDER BY 7 DESC, 4 DESC LIMIT 30;
from datetime import datetime, timedelta

results = (
    ExperimentResult.select(
        ExperimentResult.param_list_id,
        ExperimentResult.acc_test,
        ExperimentResult.fit_score,
        ExperimentResult.global_score_no_weak_acc,
        ExperimentResult.amount_of_user_asked_queries,
        ExperimentResult.classifier,
        ExperimentResult.global_score_no_weak_acc,
        ExperimentResult.amount_of_user_asked_queries,
        ExperimentResult.dataset_name,
        ExperimentResult.test_fraction,
        ExperimentResult.sampling,
        ExperimentResult.cluster,
        ExperimentResult.nr_queries_per_iteration,
        ExperimentResult.with_uncertainty_recommendation,
        ExperimentResult.with_cluster_recommendation,
        ExperimentResult.uncertainty_recommendation_certainty_threshold,
        ExperimentResult.uncertainty_recommendation_ratio,
        ExperimentResult.cluster_recommendation_minimum_cluster_unity_size,
        ExperimentResult.cluster_recommendation_ratio_labeled_unlabeled,
        ExperimentResult.allow_recommendations_after_stop,
        ExperimentResult.stopping_criteria_uncertainty,
        ExperimentResult.stopping_criteria_acc,
        ExperimentResult.stopping_criteria_std,
        ExperimentResult.experiment_run_date,
    )
    .where(
        (ExperimentResult.amount_of_user_asked_queries < 1000)
        & (ExperimentResult.dataset_name == "dwtc")
        & (
            ExperimentResult.experiment_run_date > (datetime(2020, 3, 24, 14, 0))
        )  # no stopping criterias
    )
    .order_by(
        # ExperimentResult.id_field.desc(),
        ExperimentResult.acc_test.desc(),
    )
    .limit(20)
)

# INTERESSANT: selbst wenn es keine Einschränkung bei der Berechnung auf weak/no_weak gibt werden Cluster verwendet!


table = []
id = 0
for result in results:
    data = {**{"id": id}, **vars(result)["__data__"]}
    # data["param_list_id"] = data["__data__"]["param_list_id"]
    # del data["__data__"]
    # del data["_dirty"]
    # del data["__rel__"]

    # data = {**data, **vars(one_param_list_id_result)["__data__"]}

    table.append(data)
    id += 1

display(HTML(tabulate(table, headers="keys", tablefmt="html")))

id,param_list_id,acc_test,fit_score,global_score_no_weak_acc,amount_of_user_asked_queries,classifier,dataset_name,test_fraction,sampling,cluster,nr_queries_per_iteration,with_uncertainty_recommendation,with_cluster_recommendation,uncertainty_recommendation_certainty_threshold,uncertainty_recommendation_ratio,cluster_recommendation_minimum_cluster_unity_size,cluster_recommendation_ratio_labeled_unlabeled,allow_recommendations_after_stop,stopping_criteria_uncertainty,stopping_criteria_acc,stopping_criteria_std,experiment_run_date
0,d921696e7416e1b722c2d6c1e9770fb7,0.854967,0.743153,0.759661,990,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,True,True,0.695397,0.1,0.689617,0.983513,True,1,1,1,2020-03-29 21:31:34.666743
1,ed4e8249c91e1cc8e04156f4af2908a9,0.853929,0.746724,0.715835,972,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,True,True,0.75,0.0001,0.98,0.88,False,1,1,1,2020-05-05 12:04:35.183339
2,10eab83d009a269823aa3fc02929a7e2,0.853236,0.743161,0.747801,987,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,True,False,0.75,0.0001,0.63,0.54,False,1,1,1,2020-04-14 18:05:08.387833
3,938d53967512b642ec453bdf6fe5eabb,0.850467,0.752341,0.733389,940,RF,dwtc,0.5,uncertainty_lc,dummy,10,True,True,0.6,0.001,0.92,0.76,True,1,1,1,2020-04-17 10:58:44.807415
4,ef5b9f672e9dc45fafa882441875ecf9,0.850121,0.74439,0.734901,976,RF,dwtc,0.5,uncertainty_lc,dummy,10,True,False,0.69,0.001,0.81,0.92,False,1,1,1,2020-04-15 05:44:12.302985
5,73e21df55d4bde23c726f1645ae7b925,0.849775,0.745132,0.750837,972,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,True,True,0.549005,0.1,0.908151,0.874746,True,1,1,1,2020-03-27 13:42:50.567183
6,d0ad6b4258e24e0bb9705a9ab726374a,0.849083,0.772828,0.737515,840,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,True,True,0.59,0.1,0.81,0.67,False,1,1,1,2020-04-27 04:04:36.356451
7,6d39f02cba759635417aaceeb8dcdd9f,0.849083,0.739599,0.708223,996,RF,dwtc,0.5,uncertainty_lc,dummy,10,True,True,0.7,0.01,0.55,0.84,False,1,1,1,2020-04-29 07:34:38.126077
8,a253c11a2c484f6e0ba0db03027c6906,0.848737,0.755943,0.733149,920,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,True,True,0.69,0.0001,0.5,0.89,False,1,1,1,2020-05-04 03:16:38.245939
9,78e69972ef2485da49e9bdc7f7d40695,0.848737,0.75509,0.736342,924,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,True,True,0.75,0.001,0.76,0.77,False,1,1,1,2020-04-28 18:10:23.693421


In [48]:
# SELECT id_field, param_list_id, dataset_path, start_set_size as sss, sampling, cluster, allow_recommendations_after_stop as SA, stopping_criteria_uncertainty as SCU, stopping_criteria_std as SCS, stopping_criteria_acc as SCA, amount_of_user_asked_queries as "#q", acc_test, fit_score, global_score_norm, thread_id, end_time from experimentresult where param_list_id='31858014d685a3f1ba3e4e32690ddfc3' order by end_time, fit_score desc, param_list_id;
loaded_data = {}


def pre_fetch_data(top_n=0):
    best_param_list_id = table[top_n]["param_list_id"]

    results = (
        ExperimentResult.select()
        .where(ExperimentResult.param_list_id == best_param_list_id)
        .order_by(ExperimentResult.dataset_name)
    )

    loaded_data[top_n] = []
    for result in results:
        loaded_data[top_n].append(result)
    print("Loaded Top " + str(top_n) + " data")


pre_fetch_data(0)

Loaded Top 0 data


In [49]:
def visualise_top_n(top_n=0):
    charts = []

    alt.renderers.enable("html")

    for result in loaded_data[top_n][:]:
        metrics = loads(result.metrics_per_al_cycle)
        test_data_metrics = [
            metrics["test_data_metrics"][0][f][0]["weighted avg"]
            for f in range(0, len(metrics["test_data_metrics"][0]))
        ]
        test_acc = [
            metrics["test_data_metrics"][0][f][0]["accuracy"]
            for f in range(0, len(metrics["test_data_metrics"][0]))
        ]

        data = pd.DataFrame(
            {
                "iteration": range(0, len(metrics["all_unlabeled_roc_auc_scores"])),
                "all_unlabeled_roc_auc_scores": metrics["all_unlabeled_roc_auc_scores"],
                "query_length": metrics["query_length"],
                "recommendation": metrics["recommendation"],
                "query_strong_accuracy_list": metrics["query_strong_accuracy_list"],
                "f1": [i["f1-score"] for i in test_data_metrics],
                "test_acc": test_acc,
                #'asked_queries': [sum(metrics['query_length'][:i]) for i in range(0, len(metrics['query_length']))],
            }
        )

        # bar width
        data["asked_queries"] = data["query_length"].cumsum()
        data["asked_queries_end"] = data["asked_queries"].shift(fill_value=0)

        # print(data[['asked_queries', 'query_length']])

        data["recommendation"] = data["recommendation"].replace(
            {
                "A": "Oracle",
                "C": "Weak Cluster",
                "U": "Weak Certainty",
                "G": "Ground Truth",
            }
        )

        # data = data[:100]

        # calculate global score OHNE

        chart = (
            alt.Chart(data)
            .mark_rect(
                # point=True,
                # line=True,
                # interpolate='step-after',
            )
            .encode(
                x=alt.X("asked_queries_end", title="#asked queries (weak and oracle)"),
                x2="asked_queries",
                color=alt.Color("recommendation", scale=alt.Scale(scheme="tableau10")),
                tooltip=[
                    "iteration",
                    "f1",
                    "test_acc",
                    "all_unlabeled_roc_auc_scores",
                    "query_strong_accuracy_list",
                    "query_length",
                    "recommendation",
                ],
                # scale=alt.Scale(domain=[0,1])
            )
            .properties(title=result.dataset_name)
            .interactive()
        )
        charts.append(
            alt.hconcat(
                chart.encode(
                    alt.Y(
                        "all_unlabeled_roc_auc_scores", scale=alt.Scale(domain=[0, 1])
                    )
                ).properties(title=result.dataset_name + ": roc_auc"),
                # chart.encode(alt.Y('f1', scale=alt.Scale(domain=[0,1]))).properties(title=result.dataset_name + ': f1'),
                chart.encode(
                    alt.Y("test_acc", scale=alt.Scale(domain=[0, 1]))
                ).properties(title=result.dataset_name + ": test_acc"),
            )
        )

    return alt.vconcat(*charts).configure()


# visualise_top_n(0)

In [50]:
def compare_data(datasets):
    charts = []

    alt.renderers.enable("html")
    all_data = pd.DataFrame()

    for i, dataset in enumerate(datasets):
        for result in dataset:
            if result.dataset_name != "dwtc":
                continue
            metrics = loads(result.metrics_per_al_cycle)
            test_data_metrics = [
                metrics["test_data_metrics"][0][f][0]["weighted avg"]
                for f in range(0, len(metrics["test_data_metrics"][0]))
            ]
            test_acc = [
                metrics["test_data_metrics"][0][f][0]["accuracy"]
                for f in range(0, len(metrics["test_data_metrics"][0]))
            ]

            data = pd.DataFrame(
                {
                    "iteration": range(0, len(metrics["all_unlabeled_roc_auc_scores"])),
                    "all_unlabeled_roc_auc_scores": metrics[
                        "all_unlabeled_roc_auc_scores"
                    ],
                    "query_length": metrics["query_length"],
                    "recommendation": metrics["recommendation"],
                    "query_strong_accuracy_list": metrics["query_strong_accuracy_list"],
                    "f1": [i["f1-score"] for i in test_data_metrics],
                    "test_acc": test_acc,
                    "top_n": str(i),
                    #'asked_queries': [sum(metrics['query_length'][:i]) for i in range(0, len(metrics['query_length']))],
                }
            )

            # bar width
            data["asked_queries"] = data["query_length"].cumsum()
            data["asked_queries_end"] = data["asked_queries"].shift(fill_value=0)

            # print(data[['asked_queries', 'query_length']])

            data["recommendation"] = data["recommendation"].replace(
                {
                    "A": "Oracle",
                    "C": "Weak Cluster",
                    "U": "Weak Certainty",
                    "G": "Ground Truth",
                }
            )

            all_data = pd.concat([all_data, data])

        # data = data[:100]

        # calculate global score OHNE

    points = (
        alt.Chart(all_data,)
        .mark_point()
        .encode(
            x="asked_queries:Q",
            y="test_acc:Q",
            shape="recommendation:N",
            color="recommendation:N",
        )
    )

    lines = (
        alt.Chart(all_data,)
        .mark_line(interpolate="step-before")
        .encode(
            x=alt.X(
                "asked_queries:Q", scale=alt.Scale(domain=[-1, 2900], type="linear")
            ),
            y=alt.Y("test_acc:Q", scale=alt.Scale(domain=[0.3, 0.82], type="linear")),
            color="top_n:N",
            # shape="top_n",
            # strokeDash="top_n",
            # shape="recommendation",
            # color="recommendation:N",
        )
    )

    return (
        alt.layer(points, lines)
        .resolve_scale(color="independent", shape="independent")
        .configure_legend(
            orient="bottom-right",
            padding=10,
            fillColor="#f1f1f1",
            labelOpacity=0.9,
            labelOverlap=True,
            # opacity=0.6,
        )
        .properties(title="Comparison of ")
        .interactive()
    )


# compare_data(
#    [
#        loaded_data[0],
#        loaded_data[1],
#        # loaded_data[2],
#    ]  # , loaded_data[3], loaded_data[4]]
# )

In [51]:
results = (
    ExperimentResult.select(
        ExperimentResult.amount_of_user_asked_queries, ExperimentResult.acc_test,
    ).where(
        # (ExperimentResult.stopping_criteria_acc == 1)
        # & (ExperimentResult.stopping_criteria_std == 1)
        # & (ExperimentResult.stopping_criteria_uncertainty == 1)
        # & (ExperimentResult.sampling == "random")
        #  & (ExperimentResult.cluster == "random")
        # & (ExperimentResult.amount_of_user_asked_queries == 10)
        # & (ExperimentResult.with_cluster_recommendation == True)
        # & (ExperimentResult.with_uncertainty_recommendation == True)
        (ExperimentResult.amount_of_user_asked_queries < 10)
        & (ExperimentResult.dataset_name == "dwtc")
        & (
            ExperimentResult.experiment_run_date > (datetime(2020, 3, 24, 14, 0))
        )  # no stopping criterias
    )
    #  .group_by(ExperimentResult.amount_of_user_asked_queries)
    .order_by(
        ExperimentResult.amount_of_user_asked_queries.desc(),
        ExperimentResult.acc_test.desc()
        #  peewee.fn.MAX(ExperimentResult.acc_test)
        #  peewee.fn.COUNT(ExperimentResult.).desc(),
        #  peewee.fn.AVG(ORDER_BY).desc(),
    )
    #  .limit(10)
)
data = []
for result in results:
    data.append((result.amount_of_user_asked_queries, result.acc_test))
    #  print("{}\t{}".format(result.amount_of_user_asked_queries, result.max))
pd.DataFrame(data)

,0,1
0,9,0.557979
1,9,0.549325
2,9,0.527864
3,9,0.519211
4,9,0.515057
...,...,...
21557,0,0.085843
21558,0,0.081343
21559,0,0.079612
21560,0,0.077882


In [52]:
def better_results_top(top_n, budget, weak_clust, weak_cert):
    # select best result for budget of 1500 without WS
    results = (
        ExperimentResult.select(
            ExperimentResult.param_list_id,
            peewee.fn.AVG(ExperimentResult.fit_score).alias("avg_fit_score"),
            peewee.fn.STDDEV(ExperimentResult.fit_score).alias("stddev_fit_score"),
            peewee.fn.AVG(ExperimentResult.global_score_no_weak_acc).alias(
                "avg_global_score"
            ),
            peewee.fn.STDDEV(ExperimentResult.global_score_no_weak_acc).alias(
                "stddev_global_score"
            ),
            peewee.fn.AVG(ExperimentResult.amount_of_user_asked_queries).alias(
                "avg_amount_oracle"
            ),
            peewee.fn.STDDEV(ExperimentResult.amount_of_user_asked_queries).alias(
                "std_amount_oracle"
            ),
            peewee.fn.COUNT(ExperimentResult.param_list_id).alias("count"),
        )
        .where(
            (ExperimentResult.amount_of_user_asked_queries < budget)
            & (ExperimentResult.dataset_name == "dwtc")
            # & (ExperimentResult.experiment_run_date > (datetime(2020, 3, 24, 14, 0)))
            # & (ExperimentResult.experiment_run_date > (datetime(2020, 5, 8, 9, 20)))
            & (ExperimentResult.with_cluster_recommendation == weak_clust)
            & (ExperimentResult.with_uncertainty_recommendation == weak_cert)
            # & (peewee.fn.COUNT(ExperimentResult.id_field) == 3)
            # no stopping criterias
        )
        .group_by(ExperimentResult.param_list_id)
        .order_by(
            peewee.fn.COUNT(ExperimentResult.id_field).desc(),
            peewee.fn.AVG(ExperimentResult.acc_test).desc(),
        )
        .limit(1)
        .offset(top_n)
    )

    table = []
    id = 0
    for result in results:
        data = {**{"id": id}, **vars(result)}
        data["param_list_id"] = data["__data__"]["param_list_id"]
        del data["__data__"]
        del data["_dirty"]
        del data["__rel__"]
        table.append(data)
        id += 1

    display(HTML(tabulate(table, headers="keys", tablefmt="html")))

    best_param_list_id = table[0]["param_list_id"]

    results = (
        ExperimentResult.select()
        .where(ExperimentResult.param_list_id == best_param_list_id)
        .order_by(ExperimentResult.dataset_name)
    )

    loaded_data[0] = []
    for result in results:
        loaded_data[0].append(result)

    for result in loaded_data[0][:]:
        metrics = loads(result.metrics_per_al_cycle)
        test_data_metrics = [
            metrics["test_data_metrics"][0][f][0]["weighted avg"]
            for f in range(0, len(metrics["test_data_metrics"][0]))
        ]
        test_acc = [
            metrics["test_data_metrics"][0][f][0]["accuracy"]
            for f in range(0, len(metrics["test_data_metrics"][0]))
        ]

        data = pd.DataFrame(
            {
                "iteration": range(0, len(metrics["all_unlabeled_roc_auc_scores"])),
                "all_unlabeled_roc_auc_scores": metrics["all_unlabeled_roc_auc_scores"],
                "query_length": metrics["query_length"],
                "recommendation": metrics["recommendation"],
                "query_strong_accuracy_list": metrics["query_strong_accuracy_list"],
                "f1": [i["f1-score"] for i in test_data_metrics],
                "test_acc": test_acc,
                "fit_score": result.fit_score,
                #'asked_queries': [sum(metrics['query_length'][:i]) for i in range(0, len(metrics['query_length']))],
            }
        )
        data["acc_diff"] = data["test_acc"] - data["test_acc"].shift(1)
    display(
        HTML(
            tabulate(
                data.groupby(["recommendation"]).sum(), headers="keys", tablefmt="html"
            )
        )
    )


better_results_top(0, 211, False, False)
better_results_top(0, 210, True, False)
better_results_top(0, 210, False, True)
for i in range(1, 10):
    better_results_top(i, 210, True, True)
# data

id,avg_fit_score,stddev_fit_score,avg_global_score,stddev_global_score,avg_amount_oracle,std_amount_oracle,count,param_list_id
0,0.864919,0.0038225,0.605071,0.0148437,210,0,3,9be1af7172eb7d3da78ed4e86a803e29


recommendation,iteration,all_unlabeled_roc_auc_scores,query_length,query_strong_accuracy_list,f1,test_acc,fit_score,acc_diff
A,231,18.8162,210,0,15.0569,15.2406,18.2076,0.336795
G,0,0.759992,4,0,0.435688,0.477328,0.867027,0


id,avg_fit_score,stddev_fit_score,avg_global_score,stddev_global_score,avg_amount_oracle,std_amount_oracle,count,param_list_id
0,0.834726,0.00667042,0.455027,0.0419509,202,2,3,ea5bee42f65c794983c9717673a572da


recommendation,iteration,all_unlabeled_roc_auc_scores,query_length,query_strong_accuracy_list,f1,test_acc,fit_score,acc_diff
A,1217,38.4039,201,0,25.2486,26.334,37.9043,0.26999
C,161,5.88771,49,7,3.66803,3.86154,5.89623,0.044306
G,0,0.684266,4,0,0.434149,0.455175,0.842318,0


id,avg_fit_score,stddev_fit_score,avg_global_score,stddev_global_score,avg_amount_oracle,std_amount_oracle,count,param_list_id
0,0.830242,0.0100828,0.452412,0.0474001,204,3,3,0b0afb9076c37bab26a334f0e0b6c973


...

recommendation,iteration,all_unlabeled_roc_auc_scores,query_length,query_strong_accuracy_list,f1,test_acc,fit_score,acc_diff
A,1246,39.9707,203,0,29.9082,30.1004,38.8538,0.348217
C,80,3.36022,34,4,2.3707,2.36691,3.3067,-0.0183454
G,0,0.777595,4,0,0.407235,0.41433,0.826676,0


id,avg_fit_score,stddev_fit_score,avg_global_score,stddev_global_score,avg_amount_oracle,std_amount_oracle,count,param_list_id
0,0.819904,0.0293857,0.432743,0.0871254,201,0,3,c9a023caa58b0cbb3ddecf49857fec4c


recommendation,iteration,all_unlabeled_roc_auc_scores,query_length,query_strong_accuracy_list,f1,test_acc,fit_score,acc_diff
A,1113,38.5901,202,0,29.1946,29.6224,37.9259,0.370024
G,0,0.697861,4,0,0.363241,0.393562,0.842798,0
U,63,2.61628,179,3,1.96335,1.96366,2.52839,0.00692281


id,avg_fit_score,stddev_fit_score,avg_global_score,stddev_global_score,avg_amount_oracle,std_amount_oracle,count,param_list_id
0,0.818851,0.0325753,0.494396,0.0454813,204,4,3,88ce114ed9d7bd1a991ddfeee0cfd3b2


recommendation,iteration,all_unlabeled_roc_auc_scores,query_length,query_strong_accuracy_list,f1,test_acc,fit_score,acc_diff
A,1262,35.5666,202,0,26.5555,27.1118,36.2581,0.442714
C,391,11.4161,156,14,8.37334,8.54621,11.805,0.0335756
G,0,0.625125,4,0,0.331415,0.294912,0.843212,0


In [54]:
#  SELECT param_list_id, avg(fit_score), stddev(fit_score), avg(global_score), stddev(global_score), avg(start_set_size) as sss, count(*) FROM experimentresult WHERE start_set_size = 1 GROUP BY param_list_id ORDER BY 7 DESC, 4 DESC LIMIT 30;
from datetime import datetime, timedelta

# & (ExperimentResult.experiment_run_date > (datetime(2020, 3, 24, 14, 0))) # no stopping criterias
#  & (ExperimentResult.experiment_run_date > (datetime(2020, 3, 30, 12, 23))) # optics


results = (
    ExperimentResult.select(
        peewee.fn.COUNT(ExperimentResult.id_field).alias("count"),
        ExperimentResult.param_list_id,
        peewee.fn.AVG(ExperimentResult.fit_score).alias("avg_fit_score"),
        peewee.fn.STDDEV(ExperimentResult.fit_score).alias("stddev_fit_score"),
        peewee.fn.AVG(ExperimentResult.global_score_no_weak_acc).alias(
            "avg_global_score"
        ),
        peewee.fn.STDDEV(ExperimentResult.global_score_no_weak_acc).alias(
            "stddev_global_score"
        ),
        peewee.fn.AVG(ExperimentResult.amount_of_user_asked_queries).alias(
            "avg_amount_oracle"
        ),
        peewee.fn.STDDEV(ExperimentResult.amount_of_user_asked_queries).alias(
            "std_amount_oracle"
        ),
        # peewee.fn.AVG(ExperimentResult.start_set_size).alias("sss"),
    )
    .where(
        (ExperimentResult.amount_of_user_asked_queries < 211)
        & (ExperimentResult.dataset_name == "dwtc")
        # & (ExperimentResult.experiment_run_date > (datetime(2020, 3, 24, 14, 0)))
        # & (ExperimentResult.experiment_run_date > (datetime(2020, 5, 8, 9, 20)))
        # & (ExperimentResult.with_cluster_recommendation == True)
        # & (ExperimentResult.with_uncertainty_recommendation == True)
        # & (peewee.fn.COUNT(ExperimentResult.id_field) == 3)
        # no stopping criterias
    )
    .group_by(ExperimentResult.param_list_id)
    .order_by(
        # peewee.fn.COUNT(ExperimentResult.id_field).desc(),
        peewee.fn.AVG(ExperimentResult.acc_test).desc(),
    )
    .limit(200)
)

table = []
id = 0
for result in results:
    data = {**{"id": id}, **vars(result)}
    data["param_list_id"] = data["__data__"]["param_list_id"]
    del data["__data__"]
    del data["_dirty"]
    del data["__rel__"]

    # get one param_list_id

    one_param_list_id_result = (
        ExperimentResult.select(
            ExperimentResult.classifier,
            ExperimentResult.test_fraction,
            ExperimentResult.sampling,
            ExperimentResult.cluster,
            ExperimentResult.nr_queries_per_iteration,
            ExperimentResult.with_uncertainty_recommendation,
            ExperimentResult.with_cluster_recommendation,
            ExperimentResult.uncertainty_recommendation_certainty_threshold,
            ExperimentResult.uncertainty_recommendation_ratio,
            ExperimentResult.cluster_recommendation_minimum_cluster_unity_size,
            ExperimentResult.cluster_recommendation_ratio_labeled_unlabeled,
            ExperimentResult.allow_recommendations_after_stop,
            ExperimentResult.stopping_criteria_uncertainty,
            ExperimentResult.stopping_criteria_acc,
            ExperimentResult.stopping_criteria_std,
            ExperimentResult.experiment_run_date,
        )
        .where(ExperimentResult.param_list_id == data["param_list_id"])
        .limit(1)
    )[0]

    data = {**data, **vars(one_param_list_id_result)["__data__"]}

    table.append(data)
    id += 1

display(HTML(tabulate(table, headers="keys", tablefmt="html")))

id,count,avg_fit_score,stddev_fit_score,avg_global_score,stddev_global_score,avg_amount_oracle,std_amount_oracle,param_list_id,classifier,test_fraction,sampling,cluster,nr_queries_per_iteration,with_uncertainty_recommendation,with_cluster_recommendation,uncertainty_recommendation_certainty_threshold,uncertainty_recommendation_ratio,cluster_recommendation_minimum_cluster_unity_size,cluster_recommendation_ratio_labeled_unlabeled,allow_recommendations_after_stop,stopping_criteria_uncertainty,stopping_criteria_acc,stopping_criteria_std,experiment_run_date
0,2,0.869182,0,0.616148,0,210,0,5025949c1fd31aaf3f95f847711091b0,RF,0.5,uncertainty_max_margin,dummy,10,True,True,0.91,0.001,1,0.78,True,1,1,1,2020-05-26 06:46:06.943710
1,2,0.869182,0,0.616148,0,210,0,ab645612dcc7642b0a71a5c7f1686268,RF,0.5,uncertainty_max_margin,dummy,10,True,True,0.92,0.001,1,0.62,True,1,1,1,2020-05-26 07:11:53.139710
2,1,0.869182,,0.616148,,210,,eff7c1407a14ce1c6b1876c9c1cab936,RF,0.5,uncertainty_max_margin,dummy,10,True,True,0.91,0.001,1,0.86,True,1,1,1,2020-05-27 07:10:48.345567
3,1,0.869182,,0.616148,,210,,10382932b634e0796bd2dc3bf49c0826,RF,0.5,uncertainty_max_margin,dummy,10,True,True,0.91,0.001,1,0.62,True,1,1,1,2020-05-27 08:58:37.151998
4,2,0.869182,0,0.616148,0,210,0,3ab1bf6dbb3eb6ccaf0da01a4ec8f04f,RF,0.5,uncertainty_max_margin,dummy,10,True,True,0.92,0.0001,1,0.64,True,1,1,1,2020-05-26 08:36:24.976124
5,1,0.869182,,0.616148,,210,,7a8d353b7b1549ecf753da9cd85cf142,RF,0.5,uncertainty_max_margin,dummy,10,True,True,0.92,0.001,1,0.87,True,1,1,1,2020-05-27 13:54:39.357576
6,3,0.869182,0,0.616148,0,210,0,a40dc4eb067e1d92d64eaed08ac79790,RF,0.5,uncertainty_max_margin,dummy,10,True,True,0.91,0.001,1,0.95,True,1,1,1,2020-05-26 05:33:27.887047
7,3,0.869182,0,0.616148,0,210,0,3b2fe2f7cbf9ccb42511edb414bc08cc,RF,0.5,uncertainty_max_margin,dummy,10,True,True,0.92,0.001,1,0.72,True,1,1,1,2020-05-26 10:16:10.962942
8,3,0.869182,0,0.616148,0,210,0,2db900d6d28b12c8965ce0f64e619fba,RF,0.5,uncertainty_max_margin,dummy,10,True,True,0.91,0.001,1,0.9,True,1,1,1,2020-05-26 12:16:46.160475
9,1,0.869182,,0.616148,,210,,fb927349d8f7c8d8fb8dbabe1eca3073,RF,0.5,uncertainty_max_margin,dummy,10,True,True,0.92,0.001,1,0.73,True,1,1,1,2020-05-27 03:24:44.651018
